# Hedging and Control Variates in Monte-Carlo Simulations

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

In [ ]:
#Package Import
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
sns.set_style('darkgrid')

In [ ]:
#Import functions associated with Black-Scholes Equations

%run functions_black_scholes.py


import types

# List all functions in functions_black_scholes.py
function_list = [name for name, obj in globals().items() if isinstance(obj, types.FunctionType)]
print(function_list[1:])


### Delta Based Control Variants in Monte-Carlo Simulation


Consider a call option on a stock at time $0$ that expires at time $t$. Assume the stock path follows a risk-neutral Geometric Brownian Motion:

$$S_t = S_0e^{\left(r -\frac{\sigma^2}{2} \right)t + \sigma\sqrt{t}\mathcal{N}(0,1)}.$$


Let $0 = t_0<t_1<\cdots<t_{n-1}<t_n = t.$ Then the expected payoff of the call option discounted to time $0$ can be simulated as

$$\max(S_t-K,0)e^{-rt} - \sum_{i=1}^n (S_{t_i}- e^{r(t_i-t_{i-1})}S_{t_{i-1}})\Delta_{C_{t_{i-1}}}e^{-rt_i}.$$

Where
1) $\Delta_{C_{t_{i-1}}}$ is the rate of change of the value of the call option at time $t_{i-1}$ with respect to the price of the stock $S_{t_{i-1}}$ at time $t_{i-1}$.

2) $(S_{t_i}- e^{r(t_i-t_{i-1})S_{t_{i-1}}})\Delta_{C_{i-1}}e^{-rt_i}$ is the profit of holding $\Delta_{C_{t_{i-1}}}$ shares of stock from time $t_{i-1}$ to time $t_{i}$ discounted to time $0$.

3) Each of the "heding terms" $(S_{t_i}- e^{r(t_i-t_{i-1})S_{t_{i-1}}})\Delta_{C_{i-1}}e^{-rt_i}$ has expected value $0$ and is perfectly correlated with $C_{t_{i-1}}$ by design with respect to the price of the underlying stock.

4) Subtracting the hedging terms introduce a **control variant** into the simulation and reduces overall variance in the simulated values, therefore producing a more reliable simulation whose mean is more likely to accurately reflect the true Black-Scholes price of the call option.

In [ ]:
#Simulate 1000 payoff distributions of buying 100 call options 
#with one year to expiration and 5 Delta hedges

S0 = 35
K = 35
sigma = .5
t = 1
r = 0.04

n_sims = 1000
n_hedges = 5


#Path simulation
#Create random noise for n_sims number of paths with n_hedges steps in simulated stock movements




#time interval between each step in simulated path



#Simulate call payoffs discounted to time 0



#Simulate stock profits at each interval

## profit from start to first step discounted to time 0


## stock profits in intermediate steps


#Comparison of simulated value with Black-Scholes

print(f'Simulated-Black Scholes Price: ${np.mean(profits_hedged):.2f}  with {n_sims} simulations and \
{n_hedges} control variants and standard error: {std_err:.6f}')

print('------'*17)
print('------'*17)


print(f'Black Scholes Price of Call Option: ${bs_call(S0,K,sigma,t,r):.2f}')


### Hedging and Drift Terms

Theoretically, the expected value of a call option under Black-Scholes assumptions, with a drift term, agrees with the value of the call option under the risk-neutral model of the stock path. We can visually demonstrate how regularly hedging causes this to happen.

In [ ]:
#Compare simulated call value means with different number of hedges with nonzero drift
S0 = 35
K = 35
sigma = .5
t = 1
r = 0.04
mu = .35 #Significant drift of stock movement and corresponds to approx 42% annual return

n_sims = 1000



hedging_numbers = [1, 5, 25, 50, 100, 252, 2*252]

for n_hedges in hedging_numbers:
    


    #Comparison of simulated value with Black-Scholes

    print(f'Simulated-Black Scholes Price: ${np.mean(profits_hedged):.2f}  with {n_sims} simulations and \
{n_hedges} control variants and standard error: {std_err:.6f}')

    print('------'*17)
    print('------'*17)

    
print(f'Black Scholes Price of Call Option: ${bs_call(S0,K,sigma,t,r):.2f}')


In [ ]:
#Write function that simulates black-schole price of call option using delta-based control variants
#Stock movements are allowed to have drift

S0 = 35
K = 35
sigma = .5
t = 1
r = 0.04
mu = .35 #Drift of stock movement

n_sims = 1000
n_hedges = 50


def bs_MC_call(S0, K, sigma, r, t, mu = 0, n_sims = 2500, n_hedges = 50):
    
    """Description
    Monte-Carlo simulation of the Black-Scholes value of a call option with Delta based control variants
    
    
    Parameters:
    S0 (float): spot price
    K (float): strike price
    sigma (float): volatility
    r (float): risk-free interest rate
    t (float): time to expiration
    mu (float): Drift of log-returns
    n_sims (int): Number of simulations
    n_hedges (int): number of delta control variants at evenly spaced increments
    
    
    Return:
    np.array of simulated values of Black-Scholes value of call option
    
    Warning:
    This function is not a true simulation. The Black-Scholes equation for Delta is not simulated.
    """
    #We will adjust this function later in the notebook to be a true Monte-Carlo simulation
    
    
    
    return profits_hedged

In [ ]:
#Test Function and create histograms of simulated values

S0 = 35
K = 35
sigma = .5
t = 1
r = 0.04
mu = .35 #Drift of stock movement

n_sims = 1000

hedging_numbers = [1, 5, 25, 50, 100, 252, 2*252]

bs_price = bs_call(S0,K,sigma,t,r)

for n_hedges in hedging_numbers:
    profits_hedged = bs_MC_call(S0, K, sigma, r, t, mu, n_sims, n_hedges)
    
    
    plt.figure(figsize = (8,6))
    
    plt.hist(profits_hedged, bins = 50, alpha = .4, color = 'black', label = 'Simulated Values')
    
    plt.axvline(bs_price, label = f'Black-Scholes Price: ${bs_price:.2f}', color = 'red')
    
    plt.axvline(np.mean(profits_hedged), label = f'Simulated Mean: ${np.mean(profits_hedged):.2f}', color = 'blue')
    
    plt.legend()
    
    plt.title(f'Distribution of simulated Black-Scholes values with {n_sims} simulations and\
and {n_hedges} control variants and log-drift {mu}',size = 15)
    
    plt.show()




In [ ]:
#Simulate and plot histograms of a seller of 100 call option contracts that receives a premium above 
#Black-Scholes Price
# Look at expected profits and max loss with varying number of hedges


S0 = 35
K = 32
sigma = .45
t = 1
r = 0.04
mu = .35 #Drift of stock movement
premium = bs_call(S0,K,sigma + .08, t, r) #Increased volatility results in higher price in Black-Scholes model
num_options = 100

bs_price = bs_call(S0,K,sigma,t,r)

n_sims = 2500

hedging_numbers = [1, 5, 25, 50, 100, 252, 2*252]

for n_hedges in hedging_numbers:
    sold_calls_hedged = bs_MC_call(S0, K, sigma, r, t, mu, n_sims, n_hedges)
    
    
    profits_hedged = num_options*(premium - sold_calls_hedged)
    
    
    plt.figure(figsize = (8,6))
    
    plt.hist(profits_hedged, bins = 50, alpha = .4, color = 'black', label = 'Simulated Values')
    
    plt.axvline(num_options*(premium-bs_price), label = f'Black-Scholes Expected Profit: ${num_options*(premium-bs_price):.2f}', color = 'red')
    
    plt.axvline(np.mean(profits_hedged), label = f'Simulated Mean Profit: ${np.mean(profits_hedged):.2f}', color = 'blue')
    
    plt.axvline(np.min(profits_hedged), label = f'Max Loss: ${np.min(profits_hedged):.2f}', color = 'green')
    
    plt.legend()
    
    plt.title(f'Distribution of simulated profits {n_sims} simulations and\
and {n_hedges} Delta hedges and log-drift {mu}',size = 15)
    
    plt.show()


### Delta Simulation


To truly simulate payoffs of a seller of call options, or the Black-Scholes price of a call option. We need to simulate control variants in our simulation.

Instead of relying on the Black-Scholes formula for Deltas in our hedging process, we can estimate delta as follows: Let $\varepsilon>0$ and $C_0(S_0+\varepsilon), C_0(S_0-\varepsilon)$ be the Black-Scholes prices of call options with spot prices $S_0+\varepsilon$ and $S_0-\varepsilon$ respectively. Then

$$\Delta_{C_0}\approx \frac{C_0(S_0+\varepsilon) - C_0(S_0-\varepsilon)}{2\varepsilon}.$$

In [ ]:
### Simulate Call Option Delta

S0 = 35
K = 35
sigma = .5
t = 1
r = 0.04
mu = .35 #Drift of stock movement
delta_sims = 250






print(f'Monte-Carlo Simulated Delta: {delta_estimate:.4f}')
print('-----'*17)
print('-----'*17)
print(f'Black-Scholes Delta: {bs_call_delta(S0,K,sigma,t,r):4f}')

In [ ]:
#Write a function that simulates call option Delta

def bs_MC_call_delta(S0, K, sigma, t, r, delta_sims = int(250)):
    """Description: 
    Monte-Carlo Simulation of Black-Scholes Call Delta
    
    Parameters:
    S0 (float): spot price
    K (float): strike price
    sigma (float): volatility
    t (float): time to expiration
    r (float): risk-free interest rate
    delta_sims (int): Number of simulations
    
    Return
    float: simulated delta of call option
    
    """
    

    return 
    

In [ ]:
#test function
bs_MC_call_delta(S0,K,sigma,t,r), bs_call_delta(S0,K,sigma,t,r)

In [ ]:
#Write a function that simulates call option Delta of spot prices in an array
def bs_MC_call_delta_array(S, K, sigma, t, r, delta_sims = 250):
    """Description: 
    Monte-Carlo Simulation of Black-Scholes Call Deltas in an array
    
    Parameters:
    S (np.array of floats): spot prices
    K (float): strike price
    sigma (float): volatility
    t (float): time to expiration
    r (float): risk-free interest rate
    delta_sims (int): Number of simulations
    
    Return
    float: simulated delta of call option
    
    """
    

    return 



In [ ]:
#Test Function

bs_MC_call_delta_array(np.array([S0, S0,S0, S0]), K ,sigma, t,r, 250), bs_call_delta(S0,K,sigma, t,r)

In [ ]:
### Write a function that does a true Monte-Carlo Simulation of the Black-Scholes Call Option price 
#with Delta based control variants

def bs_MC_call(S0, K, sigma, r, t, mu = 0, n_sims = 2500, n_hedges = 50, delta_sims = 250):
    
    """Description
    Monte-Carlo simulation of the Black-Scholes value of a call option with Delta based control variants
    
    
    Parameters:
    S0 (float): spot price
    K (float): strike price
    sigma (float): volatility
    r (float): risk-free interest rate
    t (float): time to expiration
    mu (float): Drift of log-returns
    n_sims (int): Number of simulations
    n_hedges (int): number of delta control variants at evenly spaced increments
    
    
    Return:
    np.array of simulated values of Black-Scholes value of call option
    """
    
    
    
    return 


In [ ]:
### Test Function

#Test Function and create histograms of simulated values

S0 = 35
K = 35
sigma = .5
t = 1
r = 0.04
mu = .35 #Drift of stock movement
delta_sims = 500

n_sims = 1000

hedging_numbers = [1, 5, 25, 50, 100, 252, 2*252]

bs_price = bs_call(S0,K,sigma,t,r)

for n_hedges in hedging_numbers:
    profits_hedged = bs_MC_call(S0, K, sigma, r, t, mu, n_sims, n_hedges, delta_sims)
    
    
    plt.figure(figsize = (8,6))
    
    plt.hist(profits_hedged, bins = 25, alpha = .4, color = 'black', label = 'Simulated Values')
    
    plt.axvline(bs_price, label = f'Black-Scholes Price: ${bs_price:.2f}', color = 'red')
    
    plt.axvline(np.mean(profits_hedged), label = f'Simulated Mean: ${np.mean(profits_hedged):.2f}', color = 'blue')
    
    plt.legend()
    
    plt.title(f'Distribution of simulated Black-Scholes values with {n_sims} simulations and\
and {n_hedges} control variants and log-drift {mu}',size = 15)
    
    plt.show()




In [ ]:
#True Monte-Carlo Simulation of payoffs of selling call 100 options at a premium above Black-Scholes.

S0 = 35
K = 32
sigma = .45
t = 1
r = 0.04
mu = .35 #Drift of stock movement
premium = bs_call(S0,K,sigma + .15, t, r) #Increased volatility results in higher price in Black-Scholes model
num_options = 100

bs_price = bs_call(S0,K,sigma,t,r)

n_sims = 1000
delta_sims = 100

hedging_numbers = [1, 5, 25, 50, 100, 252, 2*252]

for n_hedges in hedging_numbers:
    sold_calls_hedged = bs_MC_call(S0, K, sigma, r, t, mu, n_sims, n_hedges, delta_sims)
    
    
    profits_hedged = num_options*(premium - sold_calls_hedged)
    
    
    plt.figure(figsize = (8,6))
    
    plt.hist(profits_hedged, bins = 25, alpha = .4, color = 'black', label = 'Simulated Values')
    
    plt.axvline(num_options*(premium-bs_price), label = f'Black-Scholes Expected Profit: ${num_options*(premium-bs_price):.2f}', color = 'red')
    
    plt.axvline(np.mean(profits_hedged), label = f'Simulated Mean Profit: ${np.mean(profits_hedged):.2f}', color = 'blue')
    
    plt.axvline(np.min(profits_hedged), label = f'Max Loss: ${np.min(profits_hedged):.2f}', color = 'green')
    
    plt.legend()
    
    plt.title(f'Distribution of simulated profits {n_sims} simulations and\
and {n_hedges} Delta hedges and log-drift {mu}',size = 15)
    
    plt.show()


### Custom Simulations

The function `bs_MC_call`, along with the helper functions `bs_MC_call_delta` and `bs_MC_call_delta_array`, can be easily customized to accommodate more sophisticated models. These functions support two primary types of simulations:

1. **Expected Value Estimation:**  
   Simulate the expected value of a European call option under a custom model, such as one incorporating stochastic volatility, e.g. a Heston model.

2. **Hedged Payoff Distribution:**  
   Analyze the distribution of outcomes from selling a call option for a premium and implementing a delta hedging strategy over time. This enables you to model the P&L distribution of a hedged position and assess risk-adjusted performance under the custom model.

These tools provide a flexible foundation for exploring model-driven pricing and risk management strategies in quantitative finance.
